In [1]:
#setup:
#same idea as quantum battleships, but with a deck of cards, and deal or no deal style, where you collapse and measure at the end the game
#the sooner you take a chance and measure, the higher the reward
#game setup: you first pick a card, any card randomly from a deck, face down. 
#then, you choose which ones to measure, one by one.
#as they're revealed, you get to decide which card is yours whenever, and 
#initialise NO_OF_CARDS circuits, then get some kind of inputs

from qiskit import BasicAer, IBMQ
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit.providers.ibmq import least_busy
from qiskit import IBMQ
from qiskit_rng import Generator
import getpass, random, numpy, math
from qiskit.circuit.random import random_circuit
import matplotlib.pyplot as plt

import random

#must be even; the number of cards used for the game. 
NO_OF_CARDS = 12
MEASURE_LIST = [2, 3, 7, 6]

In [2]:
def create_entangled_pairs_circuit(qbits_count, measure_list):
    qc = QuantumCircuit(QuantumRegister(NO_OF_CARDS), ClassicalRegister(NO_OF_CARDS))
    
    nos = []
    for i in range(qbits_count):
        qc.h(i)
        nos.append(i)

    #randomly entangles pairs of qubits
    while(nos):
        first = nos.pop(random.randint(0,len(nos)-1))
        second = nos.pop(random.randint(0,len(nos)-1))
        qc.cx(first, second)


    for i in measure_list:
        qc.measure(i,i)
    return qc

qc = create_entangled_pairs_circuit(NO_OF_CARDS, MEASURE_LIST)
qc.draw()

shots = 1024

job = execute(qc, backend=BasicAer.get_backend('qasm_simulator'), shots=shots)
job.result().get_counts(qc)

{'000011001000': 61,
 '000000000000': 70,
 '000001001100': 71,
 '000010001000': 66,
 '000000001000': 63,
 '000001001000': 69,
 '000000000100': 62,
 '000001000000': 62,
 '000010000100': 72,
 '000001000100': 60,
 '000011001100': 60,
 '000000001100': 60,
 '000010001100': 60,
 '000011000100': 58,
 '000011000000': 66,
 '000010000000': 64}

In [3]:
def custom_random_circuit(qubits_count, measure_list):
    circ = random_circuit(qubits_count, qubits_count)
    return circ
    
qc2 = custom_random_circuit(NO_OF_CARDS, MEASURE_LIST)
qc2.draw()
shots = 1024

#job = execute(qc, backend=BasicAer.get_backend('qasm_simulator'), shots=shots)
#job.result().get_counts(qc)

In [10]:
circ = random_circuit(NO_OF_CARDS, NO_OF_CARDS, measure = True)
circ.draw(output='mpl')

job = execute(circ, backend=BasicAer.get_backend('qasm_simulator'), shots=shots)
results = job.result().get_counts(circ)
print(max(results, key=results.get))

101101101011


In [ ]:
#trying qiskit-rng library
IBMQ.load_account()
rng_provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = rng_provider.backend.ibmq_santiago

generator = Generator(backend=backend)
output = generator.sample(num_raw_bits=32).block_until_ready()
print(output.mermin_correlator)

random_bits = output.extract()